In [ ]:
from __future__ import annotations

import os
import sys
import time
import numpy as np
import uproot

from scipy.optimize import Bounds, minimize


simDir = os.getcwd()
analysisDir = os.path.join(simDir, '..', 'Analysis')
sys.path.append(analysisDir)

from simulationClass import FIMS_Simulation
from runDataClass import runData

In [ ]:
def scanParam(parameters, startValues, stopValues, numSteps):
    """
    Executes a parameter scan by running the simulation multiple times with
    varying combinations of specified parameters.

    Executes an N-dimensional linear scan with every combination of parameters.
    If a single simulation fails, its run number is set to -1 and the scan continues.

    Args:
        parameters (list): List of strings for parameters to be scanned.
        startValues (list): List of numbers; Starting value of each parameter.
        stopValues (list): List of numbers; Ending value of each parameter.
        numSteps (list): List of ints; Number of steps within each parameter's range.

    Returns:
        list: List of the run numbers of the executed simulations.
              Empty if the parameters are not recognized.
    """

    if not (len(parameters) == len(startValues) == len(stopValues) == len(numSteps)):
        raise ValueError('Inputs must have the same length.')

    simulation = FIMS_Simulation()
    allParams = simulation.defaultParam()

    for inParam in parameters:
        if inParam not in allParams:
            print(f"Parameter '{inParam}' not able to be varied.")
            return []
    
    #Find range for each parameter
    allRanges = [
        np.linspace(start, stop, step) for start, stop, step in zip(startValues, stopValues, numSteps)
    ]

    print(f'Beginning scan through {parameters}. {math.prod(numSteps)} runs.')

    runNos = []
    for scanParams in itertools.product(*allRanges):

        #Reset parameters to defaults
        simulation.resetParam(varbose=False)
        
        #Apply current parameter combination
        for i, inParam in enumerate(parameters):
            simulation.param[inParam] = scanParams[i]

        runNo = simulation.runSimulation()
        if runNo == -1:
            print(f'Simulation failed with settings: {simulation}')
        runNos.append(runNo)

    print(f'Done scan through {parameters}. Final run number: {runNos[-1]}')
    simulation.resetParam()
        
    return runNos

In [ ]:
def scanValues(parameterDefinitions):
    """
    Executes a parameter scan by running the simulation multiple times with
    varying combinations of specified parameters.

    Executes an N-dimensional linear scan with every combination of parameters.
    If a single simulation fails, its run number is set to -1 and the scan continues.

    Args:
        parameterDefinitions (list): List of lists with the following form: 
                                     ['parameter', minValue, maxValue, stepValue]
                                     'parameter' - String of the parameter name.
                                      startValue - Number for the starting parameter value.
                                      stopValue - Number for the ending parameter value.
                                      stepValue - Integer number of steps through range.

    Returns:
        list: List of the run numbers of the executed simulations.
              Empty if the parameters are not recognized.
    """
    # Use a list comprehension to validate the input first
    if not all(len(inDefinition) == 4 for inDefinition in parameterDefinitions):
        raise ValueError("Input list elements must have the form: ['parameter', startValue, stopValue, stepValue]")
    
    # Unpack the transposed lists
    parameters, startValues, stopValues, numSteps = zip(*parameterDefinitions)
    # Convert the resulting tuples to lists
    parameters = list(parameters)
    startValues = list(startValues)
    stopValues = list(stopValues)
    numSteps = list(numSteps)

    #run the scan
    runNos = scanParam(parameters, startValues, stopValues, numSteps)
        
    return runNos

In [ ]:
# Default run of a simulation.
## Determines a start-to-finish execution time.
## Individual program times are from log files.
if __name__ == '__main__':
    defaultSim = FIMS_Simulation()
    startTime = time.monotonic()
    defaultSim.runSimulation()
    endTime = time.monotonic()
    print(f'Test run takes ~{endTime-startTime:.1f}s.')

    '''
    Last time Tanner ran this test (09/08/25):
    Total time: ~690 s
        Gmsh: ~64 s
        ElmerGrid: ~1 s   
        ElmerSolver: ~545 s
        ElmerWeighting: ~55 s
        Garfield: ~28 s
    '''
    """
    Last time James ran this test:
    Total time: ~2,827 s
       Gmsh: ~204 s
       ElmerGrid: ~1 s   
       ElmerSolver: ~878 s
       ElmerWeighting: ~50 s
       Garfield: ~1,693 s
    """

In [ ]:
def getOptimizerParam():
    """
    List of parameters and their bounds to be used in the optimization process.

    Note that parameters not listed here will be held constant.

    Returns:
        list: List with elements of the form:
              ['parameter', minimumBound, maximumBound]
    """
    params = [
        ['padLength', 40, 105],
        ['gridStandoff', 50, 200], 
        ['holeRadius', 30, 90],
    ]

    return params

In [ ]:
def runForOptimizer(simFIMS):
    """
    Runs the simulation, preserving the parameters.
    
    NOTE: This makes a copy of the parameters before running the
    simulation, as the simulation will reset them at the end.
    
    Args:
        simFIMS (FIMS_Simulation): An instance of the FIMS simulation class.
        
    Returns:
        int: The run number of the simulation that was executed.
    """
    saveParam = simFIMS.param.copy()
    runNumber = simFIMS.runSimulation()
    #Put saved parameters back
    simFIMS.param = saveParam

    return runNumber

In [ ]:
def getIBN(simFIMS):
    """
    Orchestrates the process of running a simulation and calculating
    the Ion Backflow Number (IBN) from the results.
    
    Args:
        simFIMS (FIMS_Simulation): An instance of the FIMS simulation class
                                   with its parameters set for a specific run.

    Returns:
        float: The calculated Ion Backflow Number.
    """
    saveParam = simFIMS.param.copy()
    print('\n********************************')
    print('Testing Parameters: \n', saveParam)
    print('********************************\n')
    
    # Get the minimum field ratio for 100% field transparency with the current parameters.
    if not simFIMS.findMinField(10, 1.5, 1.25):
        raise ValueError('Finding minimum field failed.')
        
    #Run full simulation - TODO: This reruns Gmsh and elmerSolver 
    '''NOTE: 
        We require the weighting field to be generated. 
        This also ensures the found min field wihtin the params passed along.
        Potential solution would be to always solve the weighing field?
        Or to do that as a last-step in findMin?'''
    runNumber = runForOptimizer(simFIMS)
    
    #Get the IBN
    simData = runData(runNumber)
    IBN = simData.getRunParameter('Average IBN')

    return IBN

In [ ]:
def objectiveForIBN(optimizerParam, simFIMS, inputList):
    """
    Objective function to optimize for minimum IBN.

    Updates the simulation's parameter dictionary using the values in the optimizer
    parameter array. Then gets the current IBN, prints the value to monitor convergence,
    and returns it for the optimizer to minimize.
    
    Args:
        optimizerParam (np.array): The flat array of parameters from the optimizer.
        simFIMS (FIMS_Simulation): An instance of the FIMS simulation.
        inputList (list): A list of parameter names, matching the order of optimizerParam.
    
    Returns:
        float: The IBN value to be minimized.
    """
    # Unpack the optimizer array into the simulation's parameter dictionary.
    for i, inParam in enumerate(inputList):
        simFIMS.param[inParam] = optimizerParam[i]
    
    # Get the Ion Backflow Number
    resultIBN = getIBN(simFIMS)
    
    #Print to monitor convergence
    print('*********************************************')
    print(f'Current IBN: {resultIBN:.4f}')
    print('*********************************************')
    
    return resultIBN


In [ ]:
def optimizeForIBN():
    """
    Runs an optimization routine to find the FIMS parameters that minimize 
    the Ion Backflow Number (IBN).

    Returns:
        dict: A dictionary containing:
            - params: Dictionary of optimal FIMS parameters.
            - IBNValue: Final minimum IBN value.
            - success: Boolean representing the success status of minimization.
    """
    simFIMS = FIMS_Simulation()

    #If setting a constant parameter different than default, do so here.
    simFIMS.param['numFieldLine'] = 5 #Field lines unnecessary for optimizer
    

    #Get optimizer parameters and bounds
    params = getOptimizerParam()
    inputList = [p[0] for p in params]
    minBounds = [p[1] for p in params]
    maxBounds = [p[2] for p in params]

    #Set bounds for variables
    optimizerBounds = Bounds(minBounds, maxBounds)

    #Set initial guess as default values
    optimizerParams = [simFIMS.param[inParam] for inParam in inputList]
    initialGuess = np.array(optimizerParams)

    print('Beginning optimization...')
    result = minimize(
        fun=objectiveForIBN,
        x0=initialGuess,
        args=(simFIMS, inputList),
        method='L-BFGS-B',
        bounds=optimizerBounds
    )
    print('----- Optimization complete. -----')

    #Put results into simulation instance
    for i, inParam in enumerate(inputList):
        simFIMS.param[inParam] = result.x[i] 

    print(f'Optimal FIMS parameters for IBN = {result.fun}:')
    print(simFIMS)

    resultVals = {
        'params': simFIMS.param, 
        'ibn_value': result.fun, 
        'success': result.success
    }

    return resultVals